In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

In [2]:
pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0.0
1,2020-01-22,no,Kosovo,0.0
2,2020-01-22,no,Kuwait,0.0
3,2020-01-22,no,Kyrgyzstan,0.0
4,2020-01-22,no,Laos,0.0


In [5]:
test_data=pd_JH_data[((pd_JH_data['country']=='Italy')|
                      (pd_JH_data['country']=='Germany'))&
                     (pd_JH_data['date']>'2020-03-20')]

In [6]:
test_data.head()

,date,state,country,confirmed
16795,2020-03-21,no,Germany,16662.0
16812,2020-03-21,no,Italy,53578.0
17078,2020-03-22,no,Germany,18610.0
17096,2020-03-22,no,Italy,59138.0
17362,2020-03-23,no,Germany,22672.0


In [7]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2022-05-13,no,25723697.0
Italy,2022-05-13,no,16993813.0


In [8]:
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope

In [9]:
test_data.groupby(['state','country']).agg(np.max)

date   confirmed
state country                       
no    Germany 2022-05-13  25723697.0
      Italy   2022-05-13  16993813.0

In [10]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [11]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state  country        
no     Germany  16795            NaN
                17078            NaN
                17362       6.427510
                17647       5.190573
                17930       6.129400
                             ...    
       Italy    238048    690.473900
                238332    457.194920
                238617    341.176608
                238902    411.708841
                239183    431.899141
Name: confirmed, Length: 1568, dtype: float64

In [12]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()

In [14]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR', 'level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,497,NaN
2,Alberta,Canada,759,NaN
3,Alberta,Canada,1065,NaN
4,Alberta,Canada,1328,NaN


In [15]:
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0.0
1,1,2020-01-22,no,Kosovo,0.0
2,2,2020-01-22,no,Kuwait,0.0
3,3,2020-01-22,no,Kyrgyzstan,0.0
4,4,2020-01-22,no,Laos,0.0


In [16]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR
0,0,2020-01-22,Alberta,Canada,0.0,NaN
1,1,2020-01-22,no,Kosovo,0.0,NaN
2,2,2020-01-22,no,Kuwait,0.0,NaN
3,3,2020-01-22,no,Kyrgyzstan,0.0,NaN
4,4,2020-01-22,no,Laos,0.0,NaN


In [17]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result
        

In [18]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [19]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0
1,1,2020-01-22,no,Kosovo,0.0,NaN,0.0
2,2,2020-01-22,no,Kuwait,0.0,NaN,0.0
3,3,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0
4,4,2020-01-22,no,Laos,0.0,NaN,0.0


In [20]:

pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
239407,no,Zimbabwe,238133,2435.104575
239408,no,Zimbabwe,238380,2147.772400
239409,no,Zimbabwe,238651,1648.715738
239410,no,Zimbabwe,238913,1631.856784
239411,no,Zimbabwe,239411,1669.536553


In [21]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
239407,239407,2022-05-13,no,Algeria,265814.0,6.645233e+04,265812.8,83065.500001
239408,239408,2022-05-13,no,Albania,275534.0,5.861411e+03,275528.0,7009.890585
239409,239409,2022-05-13,no,Afghanistan,179203.0,4.976880e+03,179213.6,3330.107807
239410,239410,2022-05-13,no,Antigua and Barbuda,7721.0,inf,7732.6,443.402299
239411,239411,2022-05-13,no,Zimbabwe,249131.0,1.018018e+03,249077.0,1669.536553


In [22]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN) 

In [23]:
pd_result_larg[pd_result_larg['country']=='Germany'].tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
238030,238030,2022-05-09,no,Germany,25406868.0,456.806638,25419767.0,426.900412
238314,238314,2022-05-10,no,Germany,25503878.0,248.348783,25492944.6,378.744625
238599,238599,2022-05-11,no,Germany,25592839.0,274.249157,25577824.0,322.628485
238884,238884,2022-05-12,no,Germany,25661838.0,323.957774,25656985.8,311.823105
239166,239166,2022-05-13,no,Germany,25723697.0,392.172553,25736147.6,324.108166


In [ ]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)